In [1]:
!python -m pip install -r ./requirements_for_cuda111.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
import os
import json
import argparse
import math
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.utils import make_grid
import torchvision.transforms as T
import torchvision.datasets
from torch.utils.data import Subset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import easydict
import pickle
import pandas as pd
import gc

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

############################### Change this term for your input dataset ###############################

numofcluster = 5
clustertype = "1st"

path = "./data/"
filename1 = path + f"ml_1m_user_mov_{clustertype}_cluster={numofcluster}.csv"
new_data = pd.read_csv(filename1)

#######################################################################################################

C:\Users\User\anaconda3\envs\diffusion\lib\site-packages\IPython\core\interactiveshell.py:3460: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(torch.__version__) # 1.9.0+cu1x1 ??
print(pl.__version__) # 0.8.5 ??

1.9.0+cu111
0.8.5


In [4]:
def make_beta_schedule(schedule, start, end, n_timestep):
    """
    입력한 scheduling 방식에 따라 forward process에서 사용되는 beta를 만들어주는 함수
    input : 
        schedule (str) : scheduling 방식
        start, end (float) : beta 1, beta T 에 해당하는 값
        n_timestep (int) : T 
    output : 
        betas (tensor (n_stepsize))
    """ 
    if schedule == "quad":
        betas = torch.linspace(start ** 0.5, end ** 0.5, n_timestep, dtype=torch.float64) ** 2
    elif schedule == 'linear':
        betas = torch.linspace(start, end, n_timestep, dtype=torch.float64)
    elif schedule == 'warmup10':
        betas = _warmup_beta(start, end, n_timestep, 0.1)
    elif schedule == 'warmup50':
        betas = _warmup_beta(start, end, n_timestep, 0.5)
    elif schedule == 'const':
        betas = end * torch.ones(n_timestep, dtype=torch.float64)
    elif schedule == 'jsd':  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1. / (torch.linspace(n_timestep, 1, n_timestep, dtype=torch.float64))
    else:
        raise NotImplementedError(schedule)

    return betas


def _warmup_beta(start, end, n_timestep, warmup_frac):
    """
    make_beta_schedule에서 schedule이 warmup10, warmup50 일 때 사용되는 함수
    warmup_time만큼 증가, 그 이후로는 end 값으로 고정
    input : 
        start, end (float) : beta 1, beta T 에 해당하는 값
        n_timestep (int) : T
        warmup_frac (float) : warmup_time의 비율
    output : 
        betas (tensor (n_stepsize))
    """
    betas               = end * torch.ones(n_timestep, dtype=torch.float64)
    warmup_time         = int(n_timestep * warmup_frac)
    betas[:warmup_time] = torch.linspace(start, end, warmup_time, dtype=torch.float64)

    return betas


def normal_kl(mean1, logvar1, mean2, logvar2):
    """
    두 normal distribution의 mean, log-variance가 주어졌을 때 kl divergence를 계산하는 함수
    input:
        mean1, logvar1, mean2, logvar2 (float)
    output:
        kl-divergence (float)
    """

    kl = 0.5 * (-1.0 + logvar2 - logvar1 + torch.exp(logvar1 - logvar2) + ((mean1 - mean2) ** 2) * torch.exp(-logvar2))

    return kl


def extract(input, t, shape):
    """
    input tensor에서 첫번째 차원을 기준으로 index가 t인 값 추출, output의 shape은 shape[0], 1,...,1
    timestep에 맞는 값을 추출을 위해 사용. 예를 들어 input이 betas면, extract한 값은 beta_{t}
    input:
        input (tensor)
        t (tensor) : batch 길이 tensor, 내부는 모두 time값으로 채워져있음. tensor([1,1,...1])
        shape (tuple)
    output:
        out (tensor)
    """
    out     = torch.gather(input, 0, t.to(input.device))
    reshape = [shape[0]] + [1] * (len(shape) - 1)
    out     = out.reshape(*reshape)
    return out


def noise_like(shape, noise_fn, repeat=False):
    """
    입력한 repeat function에 따라 noise를 만들어주는 함수. 
    repeat이 false면 shape[0]을 첫번째 dim의 size로 하고, 나머지 dim의 size는 1인 noise tensor 생성. 
    repeat이 True면 shape의 dim을 가진 noise tensor 생성
    input:
        shape (tensor)
        noise_fn (function)
    output:
        shape에 맞는 noise tensor (tensor)
    
    """
    if repeat:
        resid = [1] * (len(shape) - 1)
        shape_one = (1, *shape[1:])

        return noise_fn(*shape_one).repeat(shape[0], *resid)

    else:
        return noise_fn(*shape)


def approx_standard_normal_cdf(x):
    """
    standard normal 의 CDF(x)의 approximation
    input:
        x (float)
    output:
        approx. of integral -infty to x of standard normal (float)
    """
    return 0.5 * (1.0 + torch.tanh(np.sqrt(2.0 / np.pi) * (x + 0.044715 * torch.pow(x, 3))))


def discretized_gaussian_log_likelihood(x, *, means, log_scales):
    """
    input x는 [0,255]의 int로 scaling 되어있고, 이를 [-1,1]로 rescale한 후 discretized된 log likelihood 계산하는 함수
    논문 3.3 Data scaling~ 부분 참조
    input:
        x (tensor) : x_0, generated image
        means (tensor) : mean of model at t=1
        log_scales (tensor) : 0.5*log_variance of model at t=1  
    output:
        log_probs (tensor) : 각 xi 의 log_probability를 원소로 하는 tensor
    """

    # Assumes data is integers [0, 255] rescaled to [-1, 1]
    centered_x = x - means
    inv_stdv   = torch.exp(-log_scales)
    plus_in    = inv_stdv * (centered_x + 1. / 255.)
    cdf_plus   = approx_standard_normal_cdf(plus_in)
    min_in     = inv_stdv * (centered_x - 1. / 255.)
    cdf_min    = approx_standard_normal_cdf(min_in)

    log_cdf_plus          = torch.log(torch.clamp(cdf_plus, min=1e-12))
    log_one_minus_cdf_min = torch.log(torch.clamp(1 - cdf_min, min=1e-12))
    cdf_delta             = cdf_plus - cdf_min
    log_probs             = torch.where(x < -0.999, log_cdf_plus,
                                        torch.where(x > 0.999, log_one_minus_cdf_min,
                                                    torch.log(torch.clamp(cdf_delta, min=1e-12))))

    return log_probs

In [5]:
class GaussianDiffusion(nn.Module):
    def __init__(self, betas, model_mean_type, model_var_type, loss_type):
        """
        input:
            betas (tensor) : 미리 정해진 forward process 의 variance schedule, 1- (size = (n_timesteps) )
            model_mean_type (str) : 논문 상에서는 eps 사용
            model_var_type (str) : fixedsmall, fixedlarge가 각각 논문 3.2장에서 나온 두 종류의 variance. 
            loss_type (str) : 논문 상에서는 kl 사용
        class 변수:
            self.register로 만들어진 변수들 : betas를 통해 계산할 수 있는 값들. 추후 likelihood 계산에 바로 사용하기 위해 미리 계산
        """
        super().__init__()

        betas              = betas.type(torch.float64)
        timesteps          = betas.shape[0]
        self.num_timesteps = int(timesteps)

        self.model_mean_type = model_mean_type  # xprev, xstart, eps
        self.model_var_type  = model_var_type   # learned, fixedsmall, fixedlarge
        self.loss_type       = loss_type        # kl, mse

        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, 0)
        alphas_cumprod_prev = torch.cat(
            (torch.tensor([1], dtype=torch.float64), alphas_cumprod[:-1]), 0
        )
        posterior_variance = betas * (1 - alphas_cumprod_prev) / (1 - alphas_cumprod)

        self.register("betas", betas)
        self.register("alphas_cumprod", alphas_cumprod)
        self.register("alphas_cumprod_prev", alphas_cumprod_prev)

        self.register("sqrt_alphas_cumprod", torch.sqrt(alphas_cumprod))
        self.register("sqrt_one_minus_alphas_cumprod", torch.sqrt(1 - alphas_cumprod))
        self.register("log_one_minus_alphas_cumprod", torch.log(1 - alphas_cumprod))
        self.register("sqrt_recip_alphas_cumprod", torch.rsqrt(alphas_cumprod))
        self.register("sqrt_recipm1_alphas_cumprod", torch.sqrt(1 / alphas_cumprod - 1))
        self.register("posterior_variance", posterior_variance)
        self.register("posterior_log_variance_clipped",
                      torch.log(torch.cat((posterior_variance[1].view(1, 1),
                                           posterior_variance[1:].view(-1, 1)), 0)).view(-1))
        self.register("posterior_mean_coef1", (betas * torch.sqrt(alphas_cumprod_prev) / (1 - alphas_cumprod)))
        self.register("posterior_mean_coef2", ((1 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1 - alphas_cumprod)))

    def register(self, name, tensor):
        """
        class 변수 등록을 위한 함수, class 선언 시에 변수로 설정된다.
        input:
            name (str) : 등록할 이름
            tensor (tensor) : 등록할 tensor
        """
        self.register_buffer(name, tensor.type(torch.float32))

        
    # === forward process ===
    
    
    def q_mean_variance(self, x_0, t):
        """
        q(x_t|x_0)의 mean, variance, log_variance를 return하는 함수, 논문 (4)식 참조
        input:
            x_0 (tensor): input image batch (cifar10 기준 size (32,3,32,32))
        output:
            mean, variance, log_variance : t 시점의 q의 mean, variance, log-variance (size는 x_0와 같음)
        """
        mean = extract(self.sqrt_alphas_cumprod, t, x_0.shape) * x_0
        variance = extract(1. - self.alphas_cumprod, t, x_0.shape)
        log_variance = extract(self.log_one_minus_alphas_cumprod, t, x_0.shape)
        return mean, variance, log_variance

    def q_sample(self, x_0, t, noise=None):
        """
        q(x_t|x_0) sampling 하는 함수, 논문 (4)식 참조
        input:
            x_0 (tensor): input image batch 
        output:
            t 시점의 q(x_t|x_0)을 sampling 한 tensor 
        """
        if noise is None:
            noise = torch.randn_like(x_0)
        return (extract(self.sqrt_alphas_cumprod, t, x_0.shape) * x_0
                + extract(self.sqrt_one_minus_alphas_cumprod, t, x_0.shape) * noise)
    
    def q_sample_loop(self, x_0, T, device, freq=50):
        """
        q(x_t|x_0)들의 list를 return
        t는 0~T 까지 return할 수 있으며 freq에 따라 return list의 size가 변경된다.
        input:
            freq (int): freq step마다 imglist에 append해준다. 예를 들어 freq가 50이면 x_0, x_50, x_100...을 imglist에 넣어준다
        output:
            imglist (list of tensor)
        """
        noise = torch.randn_like(x_0)
        imglist = [x_0]

        clip = lambda x_: (x_.clamp(min=-1, max=1) )
        for i in range(T+1):
            if (i+1)%freq == 0:
                imglist.append(clip(self.q_sample(x_0.to(device), torch.full((128,) , i).to(device) , noise.to(device))))
        return imglist
    
    def q_posterior_mean_variance(self, x_0, x_t, t):
        """
        q(x_(t-1)|x_t,x_0) 의 mean, variance, log_variance return하는 함수, 논문 (6), (7)식 참조
        input:
            x_0 (tensor): input image batch 
            x_t (tensor): forward process를 timestep t만큼 했을 때
        output:
            mean, var, log_var_clipped : mean, variance, log-variance of q(x_(t-1)|x_t,x_0) 
        """
        mean            = (extract(self.posterior_mean_coef1, t, x_t.shape) * x_0
                           + extract(self.posterior_mean_coef2, t, x_t.shape) * x_t)
        var             = extract(self.posterior_variance, t, x_t.shape)
        log_var_clipped = extract(self.posterior_log_variance_clipped, t, x_t.shape)

        return mean, var, log_var_clipped

    
    # === reverse process ===
    def predict_start_from_noise(self, x_t, t, noise):
        """
        mean type이 eps인 경우 x_0 예측, (12)식 변형
        """
        
        x_t = x_t.view(noise.shape)
        
        
        return (extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t
                - extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise)

    def predict_start_from_prev(self, x_t, t, x_prev):
        """
        mean type이 xprev인 경우 x_0 예측, (7)식 변형    
        """

        return (extract(1./self.posterior_mean_coef1, t, x_t.shape) * x_prev -
                extract(self.posterior_mean_coef2 / self.posterior_mean_coef1, t, x_t.shape) * x_t)
    
    def p_mean_variance(self, model, x, t, clip_denoised, return_pred_x0, condition):
        """
        reverse step model의 mean, var type에 따라 다른 mean, variance, log_variance를 return하는 함수.
        var type이 fixedsmall, fixedlarge가 각각 논문 3.2장에서 나온 두 종류의 variance. 
        mean type이 eps인 경우가 논문의 (12)식 변형, xprev인 경우가 논문의 (7)식 변경
        input:
            model (class) : 본 논문에서는 Unet model
            cleap_denoised (bool) : clipping 여부 결정 
            return_pred_x0 (bool) : 예측한 x0 return 여부 결정 
            
        output:
            mean, var, log_var, pred_x0 : mean, variance, log_variance, predicted x0 of reverse process
        """

        model_output = model(x, condition, t)

        # Learned or fixed variance?
        if self.model_var_type == 'learned':
            model_output, log_var = torch.split(model_output, 2, dim=-1)
            var                   = torch.exp(log_var)

        elif self.model_var_type in ['fixedsmall', 'fixedlarge']:

            # below: only log_variance is used in the KL computations
            var, log_var = {
                # for 'fixedlarge', we set the initial (log-)variance like so to get a better decoder log likelihood
                'fixedlarge': (self.betas, torch.log(torch.cat((self.posterior_variance[1].view(1, 1),
                                                                self.betas[1:].view(-1, 1)), 0)).view(-1)),
                'fixedsmall': (self.posterior_variance, self.posterior_log_variance_clipped),
            }[self.model_var_type]

            var     = extract(var, t, x.shape) * torch.ones_like(x)
            log_var = extract(log_var, t, x.shape) * torch.ones_like(x)
        else:
            raise NotImplementedError(self.model_var_type)

        # Mean parameterization
        _maybe_clip = lambda x_: (x_.clamp(min=-1, max=1) if clip_denoised else x_)

        if self.model_mean_type == 'xprev':
            # the model predicts x_{t-1}
            pred_x_0 = _maybe_clip(self.predict_start_from_prev(x_t=x, t=t, x_prev=model_output))
            mean     = model_output
        elif self.model_mean_type == 'xstart':
            # the model predicts x_0
            pred_x0    = _maybe_clip(model_output)
            mean, _, _ = self.q_posterior_mean_variance(x_0=pred_x0, x_t=x, t=t)
        elif self.model_mean_type == 'eps':
            # the model predicts epsilon
            pred_x0    = _maybe_clip(self.predict_start_from_noise(x_t=x, t=t, noise=model_output))
            mean, _, _ = self.q_posterior_mean_variance(x_0=pred_x0, x_t=x, t=t)
        else:
            raise NotImplementedError(self.model_mean_type)

        if return_pred_x0:
            return mean, var, log_var, pred_x0
        else:
            return mean, var, log_var



    def p_sample(self, model, x, t, noise_fn, clip_denoised=True, return_pred_x0=False, condition = torch.zeros(19)):
        """
        t가 T...1일 때 mean, log_variance 이용해 x_(t-1) sampling, 논문의 Algorithm 2 참조
        input:
            noise_fn : 논문 상에서는 z로 표기
        output:
            sample : x_(t-1)
        """

        mean, _, log_var, pred_x0 = self.p_mean_variance(model, x, t, clip_denoised, return_pred_x0=True, condition=condition)
        noise                     = noise_fn(x.shape, dtype=x.dtype).to(x.device)

        shape        = [x.shape[0]] + [1] * (x.ndim - 1)
        nonzero_mask = (1 - (t == 0).type(torch.float32)).view(*shape).to(x.device)
        sample       = mean + nonzero_mask * torch.exp(0.5 * log_var) * noise

        return (sample, pred_x0) if return_pred_x0 else sample

    @torch.no_grad()
    def p_sample_loop(self, model, shape, noise_fn=torch.randn, condition = torch.zeros(18)):
        """
        t=T부터 0까지 이전 생성 image 이용해 sampling. t=0때 sampling 완료 후 image tensor return한다. 논문의 Algorithm 2 구현
        input:
            shape (tuple) : image tensor size 
        output:
            img (tensor) : x_0에 해당하는 image tensor 
        """

        device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'
        img    = noise_fn(shape).to(device)
        img = torch.cat((img, condition),dim=1)
        for i in reversed(range(self.num_timesteps)):
            img = self.p_sample(
                model,
                img,
                torch.full((shape[0],), i, dtype=torch.int64).to(device),
                noise_fn=noise_fn,
                return_pred_x0=False,
                condition = condition
            )

        return img

    @torch.no_grad()
    def p_sample_loop_progressive(self, model, shape, device, noise_fn=torch.randn, include_x0_pred_freq=50, ret_type="img", condition= torch.zeros(19)):
        """
        p_sample_loop와 유사. 
        input:
            include_x0_pred_freq (int) : include_x0_pred_freq 번 마다 그 시점의 x_0 예측값을 기록
            ret_type (string): img 면 image 하나를 return, list면 generation process의 모든 timestep의 image list를 return. 
        output:
            x0_preds_ (tensor) : (B, num_recorded_x0_pred, C, H, W) size의 tensor
            
        """

        img = noise_fn(shape, dtype=torch.float32).to(device)

        num_recorded_x0_pred = self.num_timesteps // include_x0_pred_freq
        x0_preds_            = torch.zeros((shape[0], num_recorded_x0_pred, *shape[1:]), dtype=torch.float32).to(device)
        
        img = img.view(shape[0],-1)
        clip = lambda x_: (x_.clamp(min=-1, max=1))
        imglist = [clip(img)]
        predx0_list = []
        for i in reversed(range(self.num_timesteps)):

            # Sample p(x_{t-1} | x_t) as usual
            img, pred_x0 = self.p_sample(model=model,
                                         x=img,
                                         t=torch.full((shape[0],), i, dtype=torch.int64).to(device),
                                         noise_fn=noise_fn,
                                         return_pred_x0=True,
                                         condition = condition)
            imglist.append(clip(img))
            img = img.view(shape[0],-1)
            
            # Keep track of prediction of x0
            insert_mask = np.floor(i // include_x0_pred_freq) == torch.arange(num_recorded_x0_pred,
                                                                              dtype=torch.int32,
                                                                              device=device)

            insert_mask = insert_mask.to(torch.float32).view(1, num_recorded_x0_pred, *([1] * len(shape[1:])))
            x0_preds_   = insert_mask * pred_x0[:, None, ...] + (1. - insert_mask) * x0_preds_
            if i%include_x0_pred_freq == 0:
                predx0_list.append(pred_x0)
                print(i)
        if ret_type == "list":
            return imglist, x0_preds_, predx0_list
        elif ret_type == "img":
            return img, x0_preds_

    # === Log likelihood calculation ===

    def _vb_terms_bpd(self, model, x_0, x_t, t, clip_denoised, return_pred_x0, condition):
        """
        논문에서 사용한 loss를 구하는 함수. (5)식 참조
        t가 0인 image는 decoder의 negative log likelihood 값 return
        t가 0이 아닌 image는 KL-Divergence return    
        """

        batch_size = t.shape[0]
        true_mean, _, true_log_variance_clipped    = self.q_posterior_mean_variance(x_0=x_0,
                                                                                    x_t=x_t,
                                                                                    t=t)
        model_mean, _, model_log_variance, pred_x0 = self.p_mean_variance(model,
                                                                          x=x_t,
                                                                          t=t,
                                                                          clip_denoised=clip_denoised,
                                                                          return_pred_x0=True,
                                                                         condition = condition)

        kl = normal_kl(true_mean, true_log_variance_clipped, model_mean, model_log_variance)
        kl = torch.mean(kl.view(batch_size, -1), dim=1) / np.log(2.)

        decoder_nll = -discretized_gaussian_log_likelihood(x_0, means=model_mean, log_scales=0.5 * model_log_variance)
        decoder_nll = torch.mean(decoder_nll.view(batch_size, -1), dim=1) / np.log(2.)

        # At the first timestep return the decoder NLL, otherwise return KL(q(x_{t-1}|x_t,x_0) || p(x_{t-1}|x_t))
        output = torch.where(t == 0, decoder_nll, kl)

        return (output, pred_x0) if return_pred_x0 else output

    def training_losses(self, model, x_0, cond, t, noise=None):
        """
        loss type이 kl인 경우 위의 _vb_terms_bpd 값 return
        mse인 경우 mse loss return    
        """
        if noise is None:
            noise = torch.randn_like(x_0)

        x_t = self.q_sample(x_0=x_0, t=t, noise=noise)

        
        # Calculate the loss
        if self.loss_type == 'kl':
            # the variational bound
            losses = self._vb_terms_bpd(model=model, x_0=x_0, x_t=x_t, t=t, clip_denoised=False, return_pred_x0=False)

        elif self.loss_type == 'mse':
            # unweighted MSE
            assert self.model_var_type != 'learned'
            target = {
                'xprev': self.q_posterior_mean_variance(x_0=x_0, x_t=x_t, t=t)[0],
                'xstart': x_0,
                'eps': noise
            }[self.model_mean_type]

            model_output = model(x_t, cond, t)
            losses = torch.mean((target - model_output.view(model_output.shape[0], -1)).view(x_0.shape[0], -1)**2, dim=1)
            

        else:
            raise NotImplementedError(self.loss_type)

        return losses

    def _prior_bpd(self, x_0):
        """
        x_0의 prior distribution에 대한 복원 오차 계산하는 함수
        """

        B, T                        = x_0.shape[0], self.num_timesteps
        qt_mean, _, qt_log_variance = self.q_mean_variance(x_0,
                                                           t=torch.full((B,), T - 1, dtype=torch.int64))
        kl_prior                    = normal_kl(mean1=qt_mean,
                                                logvar1=qt_log_variance,
                                                mean2=torch.zeros_like(qt_mean),
                                                logvar2=torch.zeros_like(qt_log_variance))

        return torch.mean(kl_prior.view(B, -1), dim=1)/np.log(2.)

    @torch.no_grad()
    def calc_bpd_loop(self, model, x_0, clip_denoised, condition):
        """
        모든 timestep 에 대한 prior distribution에 대한 복원 오차 계산하는 함수    
        """

        (B, C, H, W), T = x_0.shape, self.num_timesteps

        new_vals_bt = torch.zeros((B, T))
        new_mse_bt  = torch.zeros((B, T))

        for t in reversed(range(self.num_timesteps)):

            t_b = torch.full((B, ), t, dtype=torch.int64)

            # Calculate VLB term at the current timestep
            new_vals_b, pred_x0 = self._vb_terms_bpd(model=model,
                                                     x_0=x_0,
                                                     x_t=self.q_sample(x_0=x_0, t=t_b),
                                                     t=t_b,
                                                     clip_denoised=clip_denoised,
                                                     return_pred_x0=True,
                                                     condition = condition)

            # MSE for progressive prediction loss
            new_mse_b = torch.mean((pred_x0-x_0).view(B, -1)**2, dim=1)

            # Insert the calculated term into the tensor of all terms
            mask_bt = (t_b[:, None] == torch.arange(T)[None, :]).to(torch.float32)

            new_vals_bt = new_vals_bt * (1. - mask_bt) + new_vals_b[:, None] * mask_bt
            new_mse_bt  = new_mse_bt  * (1. - mask_bt) + new_mse_b[:, None] * mask_bt

        prior_bpd_b = self._prior_bpd(x_0)
        total_bpd_b = torch.sum(new_vals_bt, dim=1) + prior_bpd_b

        return total_bpd_b, new_vals_bt, prior_bpd_b, new_mse_bt

In [6]:
## Adopted from https://github.com/rosinality/denoising-diffusion-pytorch with some minor changes.

def swish(input):
    """
    activation function. return x*sigmoid(x) 
    """
    return input * torch.sigmoid(input)


@torch.no_grad()
def variance_scaling_init_(tensor, scale=1, mode="fan_avg", distribution="uniform"):
    """
    mode에 따라 scale이 바뀌고, 그 scale대로 weight tensor를 initialize 해준다.
    """
    fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(tensor)

    if mode == "fan_in":
        scale /= fan_in

    elif mode == "fan_out":
        scale /= fan_out

    else:
        scale /= (fan_in + fan_out) / 2

    if distribution == "normal":
        std = math.sqrt(scale)

        return tensor.normal_(0, std)

    else:
        bound = math.sqrt(3 * scale)

        return tensor.uniform_(-bound, bound)


def conv2d(
    in_channel,
    out_channel,
    kernel_size,
    stride=1,
    padding=0,
    bias=True,
    scale=1,
    mode="fan_avg",
):
    """
    convolution layer return하는 함수. 
    입력한 in_channel, out_channel, kernel_size, stride, padding 에 따라 convolution layer 만든다.
    scale, mode에 따라 weight initialize 해주고 bias가 True면 0 bias를 가지도록 return.
    """
    conv = nn.Conv2d(
        in_channel, out_channel, kernel_size, stride=stride, padding=padding, bias=bias
    )

    variance_scaling_init_(conv.weight, scale, mode=mode)

    if bias:
        nn.init.zeros_(conv.bias)

    return conv


def linear(in_channel, out_channel, scale=1, mode="fan_avg"):
    """
    linear layer return 하는 함수.
    in_channel, out_channel 에 따라 linear layer 만든다
    scale, mode에 따라 weight initialize 해주고, 0 bias를 가지도록 해준다.
    """
    lin = nn.Linear(in_channel, out_channel)

    variance_scaling_init_(lin.weight, scale, mode=mode)
    nn.init.zeros_(lin.bias)

    return lin


class Swish(nn.Module):
    """
    위에서 정의한 swish 함수를 layer 형태로 만든 것
    """
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return swish(input)


class Upsample(nn.Sequential):
    """
    Upsample과 conv2d를 layer로 가지는 nn.Sequential layer
    channel을 scale factor만큼 키워준다.
    """
    def __init__(self, channel):
        layers = [
            nn.Upsample(scale_factor=2, mode="nearest"),
            conv2d(channel, channel, 3, padding=1),
        ]

        super().__init__(*layers)


class Downsample(nn.Sequential):
    """
    conv2d를 layer로 가지는 nn.Sequential layer. stride가 2라 output image의 크기가 줄어든다.
    """
    def __init__(self, channel):
        layers = [conv2d(channel, channel, 3, stride=2, padding=1)]

        super().__init__(*layers)


class ResBlock(nn.Module):
    """
    Skip connection을 사용한 network. time embedding 과 dropout을 hyperparameter로 가진다.
    """
    def __init__(self, in_channel, out_channel, time_dim, dropout):
        super().__init__()

        self.norm1 = nn.GroupNorm(32, in_channel)
        self.activation1 = Swish()
        self.conv1 = conv2d(in_channel, out_channel, 3, padding=1)

        self.time = nn.Sequential(Swish(), linear(time_dim, out_channel))

        self.norm2 = nn.GroupNorm(32, out_channel)
        self.activation2 = Swish()
        self.dropout = nn.Dropout(dropout)
        self.conv2 = conv2d(out_channel, out_channel, 3, padding=1, scale=1e-10)

        if in_channel != out_channel:
            self.skip = conv2d(in_channel, out_channel, 1)

        else:
            self.skip = None

    def forward(self, input, time):
        batch = input.shape[0]

        out = self.conv1(self.activation1(self.norm1(input)))

        out = out + self.time(time).view(batch, -1, 1, 1)

        out = self.conv2(self.dropout(self.activation2(self.norm2(out))))

        if self.skip is not None:
            input = self.skip(input)

        return out + input


class SelfAttention(nn.Module):
    """
    Self attention 적용하는 layer
    input의 channel 수 4배로 늘린 뒤 torch.chunk 로 q, k, v 나눔. 
    이때 channel의 수가 3의 배수 아닌데, query와 key의 channel수가 같고, value의 channel 수가 작음
    나머지 계산의 dimension 계산은 아래 torch.einsum에 설명
    """
    def __init__(self, in_channel):
        super().__init__()

        self.norm = nn.GroupNorm(32, in_channel)
        self.qkv = conv2d(in_channel, in_channel * 4, 1)
        self.out = conv2d(in_channel, in_channel, 1, scale=1e-10)

    def forward(self, input):
        batch, channel, height, width = input.shape

        norm = self.norm(input)
        qkv = self.qkv(norm)
        query, key, value = qkv.chunk(3, dim=1)

        attn = torch.einsum("nchw, ncyx -> nhwyx", query, key).contiguous() / math.sqrt(
            channel
        )
        attn = attn.view(batch, height, width, -1)
        attn = torch.softmax(attn, -1)
        attn = attn.view(batch, height, width, height, width)

        out = torch.einsum("nhwyx, ncyx -> nchw", attn, input).contiguous()
        out = self.out(out)

        return out + input


class TimeEmbedding(nn.Module):
    """
    transformer의 sinusoidal positional embedding 적용
    """
    def __init__(self, dim):
        super().__init__()

        self.dim      = dim
        half_dim      = self.dim // 2
        self.inv_freq = torch.exp(torch.arange(half_dim, dtype=torch.float32) * (-math.log(10000) / (half_dim - 1)))

    def forward(self, input):
        shape       = input.shape
        input       = input.view(-1).to(torch.float32)
        sinusoid_in = torch.ger(input, self.inv_freq.to(input.device))
        pos_emb     = torch.cat([sinusoid_in.sin(), sinusoid_in.cos()], dim=-1)
        pos_emb     = pos_emb.view(*shape, self.dim)
        
        return pos_emb


class ResBlockWithAttention(nn.Module):
    """
    Unet에서 기본단위로 사용되는 block
    Resblock - SelfAttention으로 이루어짐
    """
    def __init__(self, in_channel, out_channel, time_dim, dropout, use_attention=False):
        super().__init__()

        self.resblocks = ResBlock(in_channel, out_channel, time_dim, dropout)

        if use_attention:
            self.attention = SelfAttention(out_channel)

        else:
            self.attention = None

    def forward(self, input, time):
        out = self.resblocks(input, time)

        if self.attention is not None:
            out = self.attention(out)

        return out


def spatial_fold(input, fold):
    """
    h, w 를 fold만큼 나눈 dim으로 바꿔주고, channel의 dimension을 그만큼 곱함
    """
    if fold == 1:
        return input

    batch, channel, height, width = input.shape
    h_fold = height // fold
    w_fold = width // fold

    return (
        input.view(batch, channel, h_fold, fold, w_fold, fold)
        .permute(0, 1, 3, 5, 2, 4)
        .reshape(batch, -1, h_fold, w_fold)
    )


def spatial_unfold(input, unfold):
    """
    spatial_fold와 반대.
    h, w를 unfold만큼 곱한 dim으로 바꿔주고, channel의 dimension을 그만큼 나눔
    """
    if unfold == 1:
        return input

    batch, channel, height, width = input.shape
    h_unfold = height * unfold
    w_unfold = width * unfold

    return (
        input.view(batch, -1, unfold, unfold, height, width)
        .permute(0, 1, 4, 2, 5, 3)
        .reshape(batch, -1, h_unfold, w_unfold)
    )



class UNet(nn.Module):
    """
    학습에 사용할 전체 모델.
    dimension과 구조는 위의 markdown 그림 참조
    """
    def __init__(
        self,
        in_channel,
        channel,
        channel_multiplier,
        n_res_blocks,
        attn_strides,
        dropout=0,
        fold=1
    ):
        super().__init__()
        
        self.fold = fold

        time_dim = channel * 4
        
        self.enc_data = nn.Sequential(linear(numofcluster*100+18,3072))
        
        self.dec_data = nn.Sequential(linear(3072,numofcluster*100))

        n_block = len(channel_multiplier)
        

        self.time = nn.Sequential(
            TimeEmbedding(channel),
            linear(channel, time_dim),
            Swish(),
            linear(time_dim, time_dim),
        )

        down_layers   = [conv2d(in_channel * (fold ** 2), channel, 3, padding=1)]
        feat_channels = [channel]
        in_channel    = channel
        for i in range(n_block):
            for _ in range(n_res_blocks):
                channel_mult = channel * channel_multiplier[i]

                down_layers.append(
                    ResBlockWithAttention(
                        in_channel,
                        channel_mult,
                        time_dim,
                        dropout,
                        use_attention=2 ** i in attn_strides,
                    )
                )

                feat_channels.append(channel_mult)
                in_channel = channel_mult

            if i != n_block - 1:
                down_layers.append(Downsample(in_channel))
                feat_channels.append(in_channel)

        self.down = nn.ModuleList(down_layers)

        self.mid = nn.ModuleList(
            [
                ResBlockWithAttention(
                    in_channel,
                    in_channel,
                    time_dim,
                    dropout=dropout,
                    use_attention=True,
                ),
                ResBlockWithAttention(
                    in_channel, in_channel, time_dim, dropout=dropout
                ),
            ]
        )

        up_layers = []
        for i in reversed(range(n_block)):
            for _ in range(n_res_blocks + 1):
                channel_mult = channel * channel_multiplier[i]

                up_layers.append(
                    ResBlockWithAttention(
                        in_channel + feat_channels.pop(),
                        channel_mult,
                        time_dim,
                        dropout=dropout,
                        use_attention=2 ** i in attn_strides,
                    )
                )

                in_channel = channel_mult

            if i != 0:
                up_layers.append(Upsample(in_channel))

        self.up = nn.ModuleList(up_layers)

        self.out = nn.Sequential(
            nn.GroupNorm(32, in_channel),
            Swish(),
            conv2d(in_channel, 3 * (fold ** 2), 3, padding=1, scale=1e-10),
        )

    def forward(self, input, condition,time):
        
        batch = input.shape[0]
        x = torch.cat((input, condition),dim=1)
        
        x=self.enc_data(x)
        time_embed = self.time(time)
        feats = []
        out = x.view(batch, 3,32,32)
        
        out = spatial_fold(out, self.fold)
        
        for layer in self.down:
            if isinstance(layer, ResBlockWithAttention):
                out = layer(out, time_embed)

            else:
                out = layer(out)

            feats.append(out)

        for layer in self.mid:
            out = layer(out, time_embed)

        for layer in self.up:
            if isinstance(layer, ResBlockWithAttention):
                out = layer(torch.cat((out, feats.pop()), 1), time_embed)

            else:
                out = layer(out)

        out = self.out(out)
        out = spatial_unfold(out, self.fold)
        
        out = out.view(batch, 3072)
        kk = self.dec_data(out)
        
        return kk

In [7]:
class VAE(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(VAE, self).__init__()
        self.latent_dim = encoding_dim
        in_channels = input_size
        hidden_dims = [512, 256, 128]
        # Encoder
        modules = []
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Linear(in_channels, h_dim),
                    nn.LeakyReLU()
                )
            )
            in_channels = h_dim
        
        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1], self.latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1], self.latent_dim)
        
        # Decoder
        modules = []
        modules.append(
            nn.Sequential(
                nn.Linear(self.latent_dim, hidden_dims[-1]),
                nn.LeakyReLU()
            )
        )
        hidden_dims.reverse()
        
        for i in range(len(hidden_dims)-1):            
            modules.append(
                nn.Sequential(
                    nn.Linear(hidden_dims[i], hidden_dims[i+1]),
                    nn.LeakyReLU()
                )
            )
        modules.append(
            nn.Sequential(
                nn.Linear(hidden_dims[-1], input_size),
                nn.Sigmoid()
            )
        )
        self.decoder = nn.Sequential(*modules)

    def forward(self, x):
        encoded = self.encoder(x)
        mu = self.fc_mu(encoded)
        log_var = self.fc_var(encoded)
        encoded = self.reparameterize(mu, log_var)
        decoded = self.decoder(encoded)
        return encoded, decoded
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps*std + mu


df = new_data
df_dict = {}
df_encoded_dict = {}

movieid = df.iloc[:-1,0]
group_gmm_values = df.loc[df.index[-1], :]

for group_value in group_gmm_values.unique():
    group_df = df.loc[:, df.loc[df.index[-1], :] == group_value]
    
    df_dict[group_value] = group_df.iloc[:-1,:] # original data save
    
VAEs = {}
clusternum = {}
for group_num in df_dict.keys():
    if type(group_num)!=str:
        print(group_num)
        data = df_dict[group_num]
        data = data.replace(np.nan, 2.5)
        input_data = data.values
        df_dict[group_num] = input_data
        del(data)
        gc.collect()
        
        # Convert the input data to a PyTorch tensor
        print(input_data, input_data.shape)
        clusternum[group_num] = input_data.shape[1]
        input_tensor = torch.from_numpy(input_data/5).float().to("cuda")

        # Initialize the autoencoder model
        input_size = input_data.shape[1] 
        encoding_dim = input_size  
        VAE_new = VAE(input_size, 100).to("cuda")
        if torch.cuda.device_count()>1:
            VAE_new = nn.DataParallel(VAE_new)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(VAE_new.parameters(), lr=0.0001)
        
        # Training the autoencoder
        num_epochs = 100
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            _, outputs = VAE_new(input_tensor)
            loss = criterion(outputs, input_tensor)
            loss.backward()
            optimizer.step()
            if epoch%100 == 99:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')
        VAEs[group_num]=VAE_new

        
for group_num in df_dict.keys():
    if type(group_num)!=str:
        print(group_num)
        encoded, decoded = VAEs[group_num](torch.from_numpy(df_dict[group_num]/5).float().to("cuda"))
        df_encoded_dict[group_num] = encoded
        decoded = decoded*5
        print(decoded)
        print(decoded.min(), decoded.max())

0.0
[[5.  2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 ...
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]] (3706, 2141)
Epoch [100/100], Loss: 0.0011282850755378604
4.0
[[2.5 4.  4.  ... 4.  2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 ...
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]] (3706, 1457)
Epoch [100/100], Loss: 0.003153582103550434
1.0
[[2.5 5.  2.5 ... 5.  4.  3. ]
 [2.5 5.  2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 3.  1.  2.5]
 ...
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]] (3706, 916)
Epoch [100/100], Loss: 0.006165783386677504
2.0
[[2.5 2.5 2.5 ... 3.  2.5 2.5]
 [2.5 3.  2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 ...
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]
 [2.5 2.5 2.5 ... 2.5 2.5 2.5]] (3706, 988)
Epoch [100/100], Loss: 0.00448529

In [8]:
data_condition = "./data/data_delete_nogenres_1m.csv"
cond_data = pd.read_csv(data_condition)

for num, j in enumerate(movieid):
    temp_cond = torch.tensor(cond_data.loc[cond_data["movieId"]==int(j)].iloc[:,2:].values,dtype = torch.float32)
    if temp_cond.shape[0]==0:
        print(j)
    if num==0:
        condition = temp_cond
        
    else:
        condition = torch.cat((condition, temp_cond), dim=0)
        
for i in df_encoded_dict.keys():
    try:
        input_data = torch.cat((input_data, df_encoded_dict[i]),dim=1)


    except:
        input_data = df_encoded_dict[i]

print(input_data.shape)
print(condition.shape)

middle_input = (input_data.max()+input_data.min())/2
halfrange = (input_data.max()-input_data.min()) / 2

input_data = (input_data - middle_input) / halfrange

print("middle = ", middle_input, "halfrange =", halfrange)

print(input_data.max(), input_data.min())
Totaldata = (input_data, condition)

torch.Size([3706, 500])
torch.Size([3706, 18])
middle =  tensor(-0.6660, device='cuda:0', grad_fn=<DivBackward0>) halfrange = tensor(7.9600, device='cuda:0', grad_fn=<DivBackward0>)
tensor(1., device='cuda:0', grad_fn=<MaxBackward1>) tensor(-1., device='cuda:0', grad_fn=<MinBackward1>)


In [9]:
def get_train_data(conf, inputdata):
    if conf.dataset.name == 'movielens':
        data, condition = inputdata
        data = data.to("cpu").detach()
        condition = condition.detach()
        datatuple = (data, condition)
        dataset = TensorDataset(*datatuple)

        # Split the dataset into training and validation sets
        train_size = 3000
        train_set = Subset(dataset, range(train_size))
        valid_set = Subset(dataset, range(train_size, data.shape[0]))
        


    else:
        raise FileNotFoundError

    return train_set, valid_set

In [10]:
## util 복붙

class obj(object):
    """
    config가 json 형식으로 되어있습니다.
    json 파일의 데이터를 받아오는 형식을 바꿔주는 함수입니다. 
    """
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)


def accumulate(model1, model2, decay=0.9999):
    """
    아래 DDP class에서 EMA를 진행하기 위한 함수입니다.
    model1의 parameter을 기존 param*decay + model2의 param*(1-decay)로 update 합니다.
    """
    par1 = dict(model1.named_parameters())
    par2 = dict(model2.named_parameters())

    for k in par1.keys():
        par1[k].data.mul_(decay).add_(par2[k].data, alpha=1 - decay)


def samples_fn(model, diffusion, shape, condition):
    """
    sample image의 현재 범위는 -1~1입니다.
    이를 image화 시키려먼 0~1 사이의 범위로 만들어주어야 하고, 이를 수행하는 함수입니다.
    """
    samples = diffusion.p_sample_loop(model=model,
                                      shape=shape,
                                      noise_fn=torch.randn,
                                     condition = condition)
    return {
      'samples': (samples + 1)/2
    }


def progressive_samples_fn(model, diffusion, shape, device, include_x0_pred_freq=50, ret_type = "img", condition= torch.zeros(19)):
    """
    위 samples_fn과 역할이 동일합니다.
    """
    samples, progressive_samples, pro_list = diffusion.p_sample_loop_progressive(
        model=model,
        shape=shape,
        noise_fn=torch.randn,
        device=device,
        include_x0_pred_freq=include_x0_pred_freq,
        ret_type = ret_type,
        condition = condition
    )
    if ret_type == "list":
        for i in range(len(samples)):
            samples[i] = (samples[i] + 1)/2
        for j in range(len(pro_list)):
            pro_list[j] = (pro_list[j]+1)/2
        return {'samples': samples, 'progressive_samples': (progressive_samples + 1)/2, "pro_list" : pro_list}
    elif ret_type == "img":
        samples = (samples +1)/2
        return {'samples': samples, 'progressive_samples': (progressive_samples + 1)/2}


def bpd_fn(model, diffusion, x, condition):
    """
    복원 오차 계산하는 함수. diffusion.calc_bpd_loop 참고
    """
    total_bpd_b, terms_bpd_bt, prior_bpd_b, mse_bt = diffusion.calc_bpd_loop(model=model, x_0=x, clip_denoised=True, condition = condition)

    return {
      'total_bpd': total_bpd_b,
      'terms_bpd': terms_bpd_bt,
      'prior_bpd': prior_bpd_b,
      'mse': mse_bt
    }


def validate(val_loader, model, diffusion, condition):
    """
    validation dataset에 대해 bpd, mse 계산하는 함수
    """
    model.eval()
    bpd = []
    mse = []
    with torch.no_grad():
        for i, (x, y) in enumerate(iter(val_loader)):
            x       = x
            metrics = bpd_fn(model, diffusion, x, condition)

            bpd.append(metrics['total_bpd'].view(-1, 1))
            mse.append(metrics['mse'].view(-1, 1))

        bpd = torch.cat(bpd, dim=0).mean()
        mse = torch.cat(mse, dim=0).mean()

    return bpd, mse

In [11]:
class DDP(pl.LightningModule):
    
    def __init__(self, conf, inputdata):
        super().__init__()

        self.conf  = conf
        self.save_hyperparameters()
        self.input = inputdata

        self.model = UNet(self.conf.model.in_channel,
                          self.conf.model.channel,
                          channel_multiplier=self.conf.model.channel_multiplier,
                          n_res_blocks=self.conf.model.n_res_blocks,
                          attn_strides=self.conf.model.attn_strides,
                          dropout=self.conf.model.dropout,
                          fold=self.conf.model.fold
                          )
        self.ema   = UNet(self.conf.model.in_channel,
                          self.conf.model.channel,
                          channel_multiplier=self.conf.model.channel_multiplier,
                          n_res_blocks=self.conf.model.n_res_blocks,
                          attn_strides=self.conf.model.attn_strides,
                          dropout=self.conf.model.dropout,
                          fold=self.conf.model.fold
                          )

        self.betas = make_beta_schedule(schedule=self.conf.model.schedule.type,
                                        start=self.conf.model.schedule.beta_start,
                                        end=self.conf.model.schedule.beta_end,
                                        n_timestep=self.conf.model.schedule.n_timestep)

        self.diffusion = GaussianDiffusion(betas=self.betas,
                                           model_mean_type=self.conf.model.mean_type,
                                           model_var_type=self.conf.model.var_type,
                                           loss_type=self.conf.model.loss_type)
        


    def setup(self, stage):

        self.train_set, self.valid_set = get_train_data(self.conf, self.input)
        

    def forward(self, x, condition):

        return self.diffusion.p_sample_loop(self.model, x.shape, condition = condition)

    def configure_optimizers(self):

        if self.conf.training.optimizer.type == 'adam':
            optimizer = optim.Adam(self.model.parameters(), lr=self.conf.training.optimizer.lr)
        else:
            raise NotImplementedError

        return optimizer

    def training_step(self, batch, batch_nb):

        img, cond = batch[0], batch[1]
        time   = (torch.rand(img.shape[0]) * 1000).type(torch.int64).to(img.device)
        loss   = self.diffusion.training_losses(self.model, img, cond, time).mean()

        accumulate(self.ema, self.model.module if isinstance(self.model, nn.DataParallel) else self.model, 0.9999)

        tensorboard_logs = {'train_loss': loss}

        return {'loss': loss, 'log': tensorboard_logs}

    def train_dataloader(self):

        train_loader = DataLoader(self.train_set,
                                  batch_size=self.conf.training.dataloader.batch_size,
                                  shuffle=True,
                                  num_workers=self.conf.training.dataloader.num_workers,
                                  pin_memory=True,
                                  drop_last=self.conf.training.dataloader.drop_last)
        return train_loader

    def validation_step(self, batch, batch_nb):
        img, cond = batch[0], batch[1]
        time   = (torch.rand(img.shape[0]) * 1000).type(torch.int64).to(img.device)
        loss   = self.diffusion.training_losses(self.ema, img, cond, time).mean()

        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):

        avg_loss         = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}

        
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def val_dataloader(self):
        valid_loader = DataLoader(self.valid_set,
                                  batch_size=self.conf.validation.dataloader.batch_size,
                                  shuffle=False,
                                  num_workers=self.conf.validation.dataloader.num_workers,
                                  pin_memory=True,
                                  drop_last=self.conf.validation.dataloader.drop_last)

        return valid_loader

In [12]:
args = easydict.EasyDict({
        "train": False,
        "config": 'config/diffusion_movielens.json',
        "ckpt_dir": './ckpts',
        "ckpt_freq": 5,
        "n_gpu": 1,
        "model_dir": './ckpts/last.ckpt',
        "sample_dir": 'samples',
        "prog_sample_freq": 100,
        "n_samples": 100
    })

    
path_to_config = args.config
with open(path_to_config, 'r') as f:
    conf = json.load(f)

conf = obj(conf)
denoising_diffusion_model = DDP(conf, Totaldata)

In [13]:
if pl.__version__ == '0.8.5':
    checkpoint_callback = ModelCheckpoint(filepath=os.path.join(args.ckpt_dir, 'ddp_{epoch:02d}-{val_loss:.2f}'),
                                      monitor='val_loss',
                                      verbose=True,
                                      save_last=True,
                                      save_top_k=-1,
                                      save_weights_only=True,
                                      mode='auto',
                                      period=args.ckpt_freq,
                                      prefix='')
    

#     except:
#         print('The present library version of pytorch lightning is ',pl.__version__)
#         print('Please check if library requirements are satisfied')

In [14]:
trainer = pl.Trainer(fast_dev_run=False,
                     gpus=args.n_gpu,
                     max_steps=conf.training.n_iter,
                     precision=conf.model.precision,
                     gradient_clip_val=1.,
                     progress_bar_refresh_rate=1,
                     checkpoint_callback=checkpoint_callback,
                     check_val_every_n_epoch=10,
                     distributed_backend='dp'
                    )


trainer.fit(denoising_diffusion_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | UNet              | 37 M  
1 | ema       | UNet              | 37 M  
2 | diffusion | GaussianDiffusion | 0     
C:\Users\User\anaconda3\envs\diffusion\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

C:\Users\User\anaconda3\envs\diffusion\lib\site-packages\pytorch_lightning\utilities\distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


KeyboardInterrupt: 

In [15]:
torch.save(denoising_diffusion_model.state_dict(), f"./trained_ckpt/model_1m_{clustertype}={numofcluster}_VAE_sig.ckpt")

In [16]:
for num, key in enumerate(VAEs.keys()):
    torch.save(VAEs[key].state_dict(),f"./trained_ckpt/model_1m_{clustertype}={numofcluster}_VAE{key}_sig.ckpt")